In [1]:
!pip install mediapipe opencv-python


^C


In [ ]:
import cv2
import random
import mediapipe as mp
import time

# Setup MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
options = ['rock', 'paper', 'scissors']

def get_hand_gesture(landmarks):
    fingers = []
    tip_ids = [4, 8, 12, 16, 20]
    if landmarks[tip_ids[0]].x < landmarks[tip_ids[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)
    for i in range(1, 5):
        if landmarks[tip_ids[i]].y < landmarks[tip_ids[i] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    if fingers.count(1) == 0:
        return 'rock'
    elif fingers.count(1) == 5:
        return 'paper'
    elif fingers == [0, 1, 1, 0, 0]:
        return 'scissors'
    else:
        return 'unknown'

def decide_winner(player, computer):
    if player == computer:
        return "Draw"
    elif (player == 'rock' and computer == 'scissors') or \
         (player == 'scissors' and computer == 'paper') or \
         (player == 'paper' and computer == 'rock'):
        return "You Win!"
    else:
        return "You Lose!"

# Timer variables
last_round_time = time.time()
round_duration = 3  # seconds
player_move = "Waiting..."
computer_move = ""
result_text = ""

print("Game started. A new round every 3 seconds. Press ESC to exit.")

while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    current_time = time.time()

    if current_time - last_round_time > round_duration:
        # Time for a new round
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
                gesture = get_hand_gesture(handLms.landmark)
                if gesture in options:
                    player_move = gesture
                    computer_move = random.choice(options)
                    result_text = decide_winner(player_move, computer_move)
                    last_round_time = current_time  # reset timer

    # UI
    cv2.putText(frame, f'You: {player_move}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)
    cv2.putText(frame, f'Computer: {computer_move}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,255), 2)
    cv2.putText(frame, f'Result: {result_text}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    seconds_left = max(0, int(round_duration - (current_time - last_round_time)))
    cv2.putText(frame, f'Next round in: {seconds_left}s', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)

    cv2.imshow("Rock Paper Scissors - Timer Mode", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
